In [47]:
import pandas as pd
import numpy as np
import pickle

def save(model, filename='bestmodel.pickle'):
    with open('output/'+filename, 'wb') as handle:
        pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
def submit(model):
    model = model.fit(train_x,train_y)
    pred = model.predict(final_test)
    final_test['SalePrice'] = np.exp(pred)
    final_test[['Id','SalePrice']].to_csv('output/submission.csv', index=False)

In [48]:
#make train and test datasets. Splitting labels and features happens later
path_train = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/train.csv"
path_test = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/test.csv"

train = pd.read_csv(path_train)
final_test = pd.read_csv(path_test)
print(train.shape, final_test.shape)

y_col = (set(train.columns) - set(final_test.columns)).pop()
train[y_col] = np.log1p(train[y_col])

# since we use cross validation the train set does not have to be split anymore
# from sklearn.model_selection import train_test_split
# train_x, test_x, train_y, test_y = train_test_split(
#     train.drop([y_col], axis=1),train.loc[:,y_col], test_size=0.33, random_state=42) 
# ds = (train_x,test_x,train_y,test_y)
# for d in ds: print(d.shape)
train_x = train.drop([y_col], axis=1)
train_y = train.loc[:,y_col]
train_x.shape, train_y.shape


(1460, 81) (1459, 80)


((1460, 80), (1460,))

### Preprocessing

In [3]:
def correlation_ratio(categories, measurements):
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = np.sqrt(numerator/denominator)
    return eta


In [6]:
from sklearn.impute import SimpleImputer
# todo multivariate imputation, possibly with pipelines for numeric and categorical data

from sklearn.preprocessing import MinMaxScaler
# https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
# https://stackoverflow.com/questions/51237635/difference-between-standard-scaler-and-minmaxscaler/51237727
# don't know features are normal so just going with minmax scalar atm

from sklearn.preprocessing import OneHotEncoder
#https://stackoverflow.com/questions/36631163/what-are-the-pros-and-cons-between-get-dummies-pandas-and-onehotencoder-sciki
#The crux of it is that the sklearn encoder creates a function which persists and can then be applied to new data sets which use the same categorical variables, with consistent results.
# So don't use pandas get dummies, but a OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

# We create the preprocessing pipelines for both numeric and categorical data.
corr = train.corr()[y_col]
corr = corr.sort_values(ascending=False)
num_x = list(corr.index)[1:]

numeric_transformer = Pipeline(steps=[
    ('impute_num', SimpleImputer(strategy='median')),
    ('scale_num', MinMaxScaler())])

cat_x = [col for col in final_test.columns if final_test[col].dtype == 'object']
cat_x.sort(key = lambda x: -correlation_ratio(train[x],train[y_col]))

categorical_transformer = Pipeline(steps=[
    ('impute_cat', SimpleImputer(strategy='most_frequent')),
    # ('onehot_cat', OneHotEncoder(handle_unknown="ignore"))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, num_x),
        ('category', categorical_transformer, cat_x)])

### Modeling

In [7]:

# from catboost import CatBoostRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet,SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import cross_validate
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def inv_y(transformed_y):
    return np.exp(transformed_y)

from collections import namedtuple
p = namedtuple('params', ['model','hyper'])

models = {'Ridge': p(Ridge(),
                {'alpha':(0.00001,1.0,'log-uniform')}),
            'svm.SVR': p(svm.SVR(),
                {'gamma': (1e-4,0.9,'log-uniform'),
                'C': [1, 10, 100, 1000, 10000]}),
            'LinearRegression':p(LinearRegression(),
                {}),
            'Lasso':p(Lasso(),
                {'alpha':(0.00001,1.0,'log-uniform')}),
            'ElasticNet':p(ElasticNet(),
                {}),
            'KNeighborsRegressor':p(KNeighborsRegressor(),
                {}),
            'RandomForestRegressor':p(RandomForestRegressor(),
                {'n_estimators' : (1e-4,0.9,'log-uniform'),
                'max_depth' : [3, 10, 20, 40]}),
            'SGDRegressor':p(SGDRegressor(),
                {'loss' : ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                'penalty' : ['l1', 'l2', 'elasticnet'],
                'alpha' : (0.0,1000,'log-uniform'),
                'learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive'],
                'class_weight' : [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}],
                'eta0' : [1, 10, 100]}),
            'CatBoostRegressor':p(CatBoostRegressor(silent=True),
                {}),
            'xgb.XGBRegressor':p(xgb.XGBRegressor(),
                {'max_depth': [4, 16, 1],
                'min_child_weight': (1, 10, 1),
                'subsample': (0.7, 1),
                'gamma' : (0.1,0.5),
                'colsample_bytree' : (0.7,1),
                'reg_lambda' : (0,1)}),
            'lgb.LGBMRegressor':
                p(lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20),
                {}),
            'HistGradientBoostingRegressor':p(HistGradientBoostingRegressor(),
                {}),
            'LassoCV':p(LassoCV(),
                {}),
            'MLPRegressor':p(MLPRegressor(),
                {})
}

# Logistic
# penalty = ['l1', 'l2']
# C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
# class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]

def get_pipeline(model):
    return Pipeline([('preprocess', preprocessor),
                   ('model',model)])



In [62]:
for name in train_x.columns:
    if name in cat_x:
        train_x[name] = train_x[name].astype(str)
        train_x[name].apply(str)
cat_x_ind = [ind for ind,name in enumerate(train_x.columns) if name in cat_x ]
cat_x_ind = list(range(37,80))
pipe = get_pipeline(CatBoostRegressor(silent=True,one_hot_max_size=20, cat_features = cat_x_ind))
num_fold = 3
scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
# scoring is identical to 
# make_scorer(mean_squared_error,greater_is_better=False, root=False,squared=False)
# neg_mean_squared_log_error
print(f"{name:<25}train {-1 * sum(scores['train_score'])/num_fold:.3f}, test {-1 * sum(scores['test_score'])/num_fold:.3f}")

SaleCondition            train 0.033, test 0.120


In [59]:
len(num_x)

37

In [57]:
train_x['Id'].astype(str)
        

0          1
1          2
2          3
3          4
4          5
        ... 
1455    1456
1456    1457
1457    1458
1458    1459
1459    1460
Name: Id, Length: 1460, dtype: object

In [32]:
len(x[0])

80

In [23]:
def cross_val_models(to_test):
    for name in to_test:
        print(name)
        pipe = get_pipeline(models[name].model)
        # score = -1 * cross_val_score(pipe, train_x, train_y,cv=3,scoring='neg_root_mean_squared_error')
        # below is necessary for looking at train scores
        num_fold = 3
        scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
        # scoring is identical to 
        # make_scorer(mean_squared_error,greater_is_better=False, root=False,squared=False)
        # neg_mean_squared_log_error
        print(f"{name:<25}train {-1 * sum(scores['train_score'])/num_fold:.3f}, test {-1 * sum(scores['test_score'])/num_fold:.3f}")

## Hyperparameter tuning

In [51]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from aoc import timeit

def model_instance(model_name):
    model = {'model': [models[model_name].model]}
    for k,v in models[model_name].hyper.items():
        model['model__'+k] = v
    return (model, 50)

def run_model(name):
    def on_step(optim_result):
        score = opt.best_score_
        print(f"{'best score':15}{score}")
        if score > opt.train_status['current_score']:
            opt.train_status['current_score'] = score
            opt.train_status['not_improving'] = 0
        else:
            opt.train_status['not_improving'] += 1
            if opt.train_status['not_improving'] == opt.train_status['stop_thres']: return True

    opt = BayesSearchCV(
    pipe,
    [model_instance(name)],
    cv=5, 
    scoring = 'neg_root_mean_squared_error',
    return_train_score = True,
    random_state = 112 
    )
    opt.train_status = { 'current_score': -100, 'not_improving': 0, 'stop_thres' : 20}

    with timeit('fitting'+name): opt.fit(train_x,train_y, callback = on_step)
    return opt

pipe = get_pipeline(svm.SVR())
to_test = ['Lasso']
test_results = {name: run_model(name) for name in to_test}

best score     -0.22016394104958886
best score     -0.15103038400115953
best score     -0.14670236053304353
best score     -0.14670236053304353
best score     -0.1361014811618008
best score     -0.1361014811618008
best score     -0.1361014811618008
best score     -0.1352495930390204
best score     -0.1352495930390204
best score     -0.1352495930390204
best score     -0.1352495930390204
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -0.1348139379074011
best score     -

In [55]:
# test_results['Lasso'].cv_results_
test_results['Lasso'].best_params_
test_results['Lasso'].best_score_
# test_results['Lasso'].total_iterations

-0.13479551731346873

In [64]:
pipe = get_pipeline(Lasso(alpha=0.0006016868735786635))
# score = -1 * cross_val_score(pipe, train_x, train_y,cv=3,scoring='neg_root_mean_squared_error')
# below is necessary for looking at train scores
num_fold = 5
scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
# scoring is identical to 
# make_scorer(mean_squared_error,greater_is_better=False, root=False,squared=False)
# neg_mean_squared_log_error
print(f"{name:<25}train {-1 * sum(scores['train_score'])/num_fold:.4f}, test {-1 * sum(scores['test_score'])/num_fold:.4f}")

MLPRegressor             train 0.1090, test 0.1348


In [61]:
pipe = get_pipeline(Lasso(alpha=0.0006607161323268261))
# score = -1 * cross_val_score(pipe, train_x, train_y,cv=3,scoring='neg_root_mean_squared_error')
# below is necessary for looking at train scores
num_fold = 5
scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
# scoring is identical to 
# make_scorer(mean_squared_error,greater_is_better=False, root=False,squared=False)
# neg_mean_squared_log_error
print(f"{name:<25}train {-1 * sum(scores['train_score'])/num_fold:.4f}, test {-1 * sum(scores['test_score'])/num_fold:.4f}")

MLPRegressor             train 0.1107, test 0.1350


## Saving

In [ ]:
train_info = (train_x, train_y, num_x, cat_x)
with open('output/train_info.pickle', 'wb') as handle:
    pickle.dump(train_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
model = get_pipeline(CatBoostRegressor(silent=True))
submit(model)
save(model)

In [0]:
m